In [1]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [10]:
text=(open("/Users/maria/github/Dataset.txt", encoding = "utf8").read().lower())

print('Total lenght:', len(text))
chars = sorted(list(set(text.split())))
print('total words:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
print(chars)

Total lenght: 11070
total words: 756
['!!', '".', '"casa', '"nanoaventuras', '#cahsi', '#colegiales', '#compscience', '#ece', '#hackathon', '#henaac30', '#hourofcode,', '#iap', '#icom', '#inel', '#outreach', '#research', '#uprm', '#utep.', '(b-392)', '(cra-e)', '(from', '(nsrec)', '-', '/', '13th', '14,', '15,', '16,', '16-20.', '17th', '18', '18th', '19th,', '20', '2014,', '2017', '2018', '2018.', '2019', '21', '26', '26th', '26th,', '2:00', '2pm,', '31', '3pm.', '3th', '7:30', '7th', '8,', '9028', '9am', '@adrianildefonso,', 'a', 'abilities', 'able', 'about', 'academia', 'academic', 'accenture', 'accomplishments', 'achievement', 'achievements.', 'aconer', 'acronym', 'activities', 'activity', 'activity.', 'adrian', 'advanced', 'advancement,', 'advisory', 'affiliates', 'after', 'age.', 'agrimensores', 'aguada,', 'agustin', 'aid', 'aim', 'algorithms', 'all', 'also', 'also,', 'alumni', 'always', 'an', 'and', 'andrade,', 'andrés', 'annual', 'another', 'ao', 'aponte,', 'apply', 'applying',

In [9]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))


nb sequences: 3677


In [6]:

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


Vectorization...


KeyError: '\ufeff'

In [ ]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
        
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])